In [1]:
!pip install langchain PyMuPDF chromadb fuzzywuzzy transformers pandas torch langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 59.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/4

In [ ]:
#create .env and add OPENAI_API_KEY, LANGCHAIN_API_KEY

import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Load lab report PDFs
pdf_path = "/kaggle/input/labdata/137253305_bgaawavakhroaw0xoj05dgcq.pdf"  # Replace with your file path
loader = PyMuPDFLoader(pdf_path)
documents = loader.load()

# Create vector store with OpenAI embeddings
vectorstore = Chroma.from_documents(documents, embedding=OpenAIEmbeddings())

# Create retriever chain
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # Top 5 relevant chunks
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(), retriever=retriever)

# Extract lab tests from the report
query = "List all lab tests mentioned in the report."
extracted_lab_tests = qa.run(query)

# Convert extracted text into a list (assuming it's comma-separated)
lab_tests = [test.strip() for test in extracted_lab_tests.split(",")]

print("Extracted Lab Tests:", lab_tests)

<ipython-input-5-86d315bf475c>:17: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(), retriever=retriever)
<ipython-input-5-86d315bf475c>:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  extracted_lab_tests = qa.run(query)


Extracted Lab Tests: ['The lab tests mentioned in the report are:\n\n1. Lipid Profile', 'Basic', 'Serum\n2. HbA1c (Glycosylated Hemoglobin)', 'Blood']


In [9]:
import pandas as pd
from fuzzywuzzy import process

# Load ICD-11 data
icd_data = pd.read_csv("/kaggle/input/labdata/ICD_11_Codes.csv")  # Ensure this has 'Code' & 'Title'
icd_data['Title'] = icd_data['Title'].astype(str).str.lower().str.strip()
icd_titles = icd_data['Title'].tolist()

# Function to match lab tests to ICD-11
def find_icd_code(test_name):
    test_name = test_name.lower().strip()

    if not icd_titles:
        return "No ICD-11 match found"

    match = process.extractOne(test_name, icd_titles)

    if match and match[1] > 80:  # Only accept matches > 80% similarity
        best_match = match[0]
        icd_code = icd_data.loc[icd_data['Title'] == best_match, 'Code'].values
        return icd_code[0] if len(icd_code) > 0 else "No ICD-11 match found"
    
    return "No ICD-11 match found"

# Map extracted lab tests to ICD-11 codes
lab_test_to_icd = {test: find_icd_code(test) for test in lab_tests}

print("Lab Test to ICD-11 Mapping:", lab_test_to_icd)

Lab Test to ICD-11 Mapping: {'The lab tests mentioned in the report are:\n\n1. Lipid Profile': '1B10', 'Basic': 'VD03', 'Serum\n2. HbA1c (Glycosylated Hemoglobin)': '4A01.01', 'Blood': 'XA8EC5'}


In [10]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load BioBERT model
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

# Function to generate embeddings
def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :]  # Use the [CLS] token embedding

# Generate embeddings for lab tests and ICD-11 codes
embeddings_dict = {}
for test, icd_code in lab_test_to_icd.items():
    if icd_code != "No ICD-11 match found":
        test_embedding = generate_embedding(test)
        icd_embedding = generate_embedding(icd_code)
        embeddings_dict[test] = {"lab_test_embedding": test_embedding, "icd_embedding": icd_embedding}

# Print results
for test, info in embeddings_dict.items():
    print(f"Lab Test: {test} | Embedding Shape: {info['lab_test_embedding'].shape}")

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Lab Test: The lab tests mentioned in the report are:

1. Lipid Profile | Embedding Shape: torch.Size([1, 768])
Lab Test: Basic | Embedding Shape: torch.Size([1, 768])
Lab Test: Serum
2. HbA1c (Glycosylated Hemoglobin) | Embedding Shape: torch.Size([1, 768])
Lab Test: Blood | Embedding Shape: torch.Size([1, 768])


In [11]:
import pickle
import torch

# Create dictionary to store embeddings before fusion
saved_embeddings = {}

for test, info in embeddings_dict.items():
    saved_embeddings[test] = {
        "lab_test_embedding": info["lab_test_embedding"].detach().cpu(),
        "icd_embedding": info["icd_embedding"].detach().cpu(),
        "icd_code": lab_test_to_icd[test]
    }

# Save embeddings as a .pt file (PyTorch format)
torch.save(saved_embeddings, "lab_icd_embeddings.pt")

# Save embeddings as a .pkl file (Pickle format)
with open("lab_icd_embeddings.pkl", "wb") as f:
    pickle.dump(saved_embeddings, f)

print("Embeddings saved successfully!")

Embeddings saved successfully!
